In [ ]:
# 필수 패키지 설치 및 환경 설정
import sys
import os
sys.path.append(os.path.abspath('../backend'))

import openai
import sqlite3
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Tuple
import json
from dataclasses import dataclass
import time

# 실험용 추가 패키지
try:
    from sentence_transformers import SentenceTransformer
    from sklearn.metrics.pairwise import cosine_similarity
    from sklearn.feature_extraction.text import TfidfVectorizer
    import faiss
    VECTOR_LIBS_AVAILABLE = True
except ImportError:
    print("벡터 라이브러리들이 설치되지 않음. 설치하려면:")
    print("pip install sentence-transformers scikit-learn faiss-cpu")
    VECTOR_LIBS_AVAILABLE = False

@dataclass
class Recommendation:
    """추천 항목 데이터 클래스"""
    itemno: str
    process: Optional[str] = None
    location: Optional[str] = None
    equipType: Optional[str] = None
    statusCode: Optional[str] = None
    priority: Optional[str] = None
    score: float = 0.0
    work_title: Optional[str] = None
    work_details: Optional[str] = None

print("환경설정 완료!")


In [ ]:
class ExperimentalRecommendationEngine:
    """
    실험용 추천 엔진
    - 실제 프로젝트: backend/app/logic/recommender.py
    - 다양한 추천 알고리즘 및 유사도 계산 방식 실험
    
    실험 가능한 요소들:
    1. 유사도 계산 (문자열, 벡터, TF-IDF 등)
    2. 추천 알고리즘 (협업 필터링, 콘텐츠 기반 등)
    3. 성능 최적화 (캐싱, 인덱싱 등)
    """
    
    def __init__(self, similarity_method: str = "string"):
        """
        추천 엔진 초기화
        
        Args:
            similarity_method: 유사도 계산 방법
                - "string": 문자열 기반 유사도
                - "vector": 벡터 기반 유사도 (sentence-transformers)
                - "tfidf": TF-IDF 기반 유사도
        """
        self.similarity_method = similarity_method
        self.vector_model = None
        self.tfidf_vectorizer = None
        
        if similarity_method == "vector" and VECTOR_LIBS_AVAILABLE:
            self.vector_model = SentenceTransformer('jhgan/ko-sroberta-multitask')
            print("벡터 모델 로딩 완료")
        elif similarity_method == "tfidf" and VECTOR_LIBS_AVAILABLE:
            self.tfidf_vectorizer = TfidfVectorizer()
            print("TF-IDF 벡터라이저 준비 완료")
    
    def calculate_similarity(self, item1: Dict, item2: Dict) -> float:
        """
        두 항목 간의 유사도 계산 (실험용)
        
        실험 아이디어:
        1. 다양한 유사도 메트릭 비교
        2. 가중치 조정 실험
        3. 새로운 피처 추가
        """
        if self.similarity_method == "string":
            return self._calculate_string_similarity(item1, item2)
        elif self.similarity_method == "vector":
            return self._calculate_vector_similarity(item1, item2)
        elif self.similarity_method == "tfidf":
            return self._calculate_tfidf_similarity(item1, item2)
        else:
            return 0.0
    
    def _calculate_string_similarity(self, item1: Dict, item2: Dict) -> float:
        """문자열 기반 유사도 계산"""
        score = 0.0
        total_weight = 0.0
        
        # 각 필드별 가중치 (실험 시 조정 가능)
        weights = {
            'equipType': 0.4,
            'location': 0.3,
            'statusCode': 0.2,
            'priority': 0.1
        }
        
        for field, weight in weights.items():
            if item1.get(field) and item2.get(field):
                if item1[field] == item2[field]:
                    score += weight
                elif item1[field] in item2[field] or item2[field] in item1[field]:
                    score += weight * 0.5
                total_weight += weight
        
        return score / total_weight if total_weight > 0 else 0.0

# 실험용 테스트 데이터
SAMPLE_NOTIFICATIONS = [
    {"itemno": "001", "equipType": "Pressure Vessel", "location": "No.1 PE", "statusCode": "고장", "priority": "긴급작업"},
    {"itemno": "002", "equipType": "Motor Operated Valve", "location": "RFCC", "statusCode": "작동불량", "priority": "일반작업"},
    {"itemno": "003", "equipType": "Pressure Vessel", "location": "No.1 PE", "statusCode": "누설", "priority": "우선작업"},
    {"itemno": "004", "equipType": "Pump", "location": "석유제품배합/저장", "statusCode": "고장", "priority": "긴급작업"},
]

def test_recommendation_engines():
    """추천 엔진들 비교 테스트"""
    query_item = {"equipType": "Pressure Vessel", "location": "No.1 PE", "statusCode": "고장"}
    
    methods = ["string", "vector", "tfidf"]
    
    print("=== 추천 엔진 비교 테스트 ===")
    print(f"쿼리: {query_item}")
    
    for method in methods:
        print(f"\\n--- {method.upper()} 방식 ---")
        
        try:
            engine = ExperimentalRecommendationEngine(method)
            recommendations = []
            
            for item in SAMPLE_NOTIFICATIONS:
                similarity = engine.calculate_similarity(query_item, item)
                recommendations.append((item["itemno"], similarity, item))
            
            # 유사도순 정렬
            recommendations.sort(key=lambda x: x[1], reverse=True)
            
            for itemno, score, item in recommendations[:3]:
                print(f"ITEMNO {itemno}: 유사도 {score:.3f} - {item['equipType']}")
                
        except Exception as e:
            print(f"{method} 방식 오류: {e}")

print("추천 엔진 실험 코드 준비 완료!")
